In [74]:
%pip install pandas
%pip install prophet


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [75]:
from prophet import Prophet
import pandas as pd

# Data Preparation

In [76]:
df_bills = pd.read_csv('data/bills.csv')
df_venues = pd.read_csv('data/venues.csv')

In [86]:
df_bills.columns

Index(['bill_paid_at_local', 'bill_total_billed',
       'bill_total_discount_item_level', 'bill_total_gratuity',
       'bill_total_net', 'bill_total_tax', 'bill_total_voided', 'bill_uuid',
       'business_date', 'order_duration_seconds', 'order_seated_at_local',
       'order_closed_at_local', 'order_take_out_type_label', 'order_uuid',
       'payment_amount', 'payment_count', 'payment_total_tip',
       'sales_revenue_with_tax', 'venue_xref_id', 'waiter_uuid'],
      dtype='object')

In [87]:
df_venues.columns

Index(['venue_xref_id', 'concept', 'city', 'country'], dtype='object')

In [78]:
df_venues = df_venues.drop(columns='start_of_day_offset')
df_venues

,venue_xref_id,concept,city,country
0,ea74268311cfcc47d2b4c38ef08b9dab0117226a9a99a9...,FAST_FOOD,Albuquerque,US
1,e7cec41c46a9706ba1a702b368be5431913b6dc9c9da49...,FAMILY_DINING,Brooklyn,US
2,e63e79791883b0c4ac71d41eaee727932c13d6a4ec8f78...,FAST_CASUAL,Windsor,CA
3,c100388c9328b30b6bdfbea43f113d367ae9d32b03e84b...,BAR,Burnaby,CA
4,a819ee7f83f27dd3d7f44515e22aca3c26e6704ddd2a9b...,FAMILY_DINING,Tisdale,CA
...,...,...,...,...
596,cfd1fa6e8f122e93b1cb02e7b7541749335e7088095f8e...,NaN,Toronto,CA
597,75e7251298f2d62406dbfe1a11011cffdad90b27704687...,NaN,Richardson,US
598,40c38939612f59ad418569e94a075935576bff0449c7f8...,FAST_CASUAL,Guelph,CA
599,2ed5bf26505a6064d09b40bacdb6fccc9665c3de029494...,NaN,Mississauga,CA


## Data Cleaning

In [79]:
missing_venues = df_venues[~df_venues['venue_xref_id'].isin(df_bills['venue_xref_id'])]
missing_venues

# All 'venue_xref_id' in df_venues exist in 'venue_xref_id' column in df_bills

,venue_xref_id,concept,city,country


In [81]:
print(df_venues['concept'].unique())

# We will omit fine_dining restaurants since they don't do promotions
# We will omit pop_up restaurants since they don't have long-term data
concept_counts = df_venues['concept'].value_counts(dropna=False)
filtered_counts = concept_counts.loc[['FINE_DINING', 'POP_UP']]
nan_count = df_venues['concept'].isna().sum()
filtered_counts.loc['NaN'] = nan_count
print(filtered_counts)

['FAST_FOOD' 'FAMILY_DINING' 'FAST_CASUAL' 'BAR' 'CAFE' 'SPORTS_CLUB' nan
 'FINE_DINING' 'BREWERY' 'POP_UP' 'BAKERY' 'BUFFET' 'HOTEL'
 'ENTERTAINMENT_COMPLEX' 'FOOD_TRUCK']
concept
FINE_DINING     19
POP_UP           1
NaN            136
Name: count, dtype: int64


In [91]:
print(df_venues['city'].isna().sum())
# No NaN in city column

0


In [96]:
# Get venue_xref_id for rows where concept is 'FINE_DINING', 'POP_UP', or NaN
venues_to_remove = df_venues[df_venues['concept'].isin(['FINE_DINING', 'POP_UP']) | df_venues['concept'].isna()]['venue_xref_id'].unique()
# Keep only rows where venue_xref_id is NOT in venues_to_remove
df_venues = df_venues[~df_venues['venue_xref_id'].isin(venues_to_remove)]
# Remove rows from df_bills that have the filtered venue_xref_id
df_bills = df_bills[~df_bills['venue_xref_id'].isin(venues_to_remove)]
print(f"Filtered df_venues: {df_venues.shape}")
print(f"Original df_bills: {df_bills.shape}")


Filtered df_venues: (445, 4)
Original df_bills: (7121732, 20)


# ML Prediction Model